# Shoreline Forecasting Data Analysis Example

In [1]:
import pandas as pd

from datetime import datetime

from preprocessing.helpers import (
    str2int,
    optimize,
    unnesting,
    pivot_tsdf,
    format_tsdf,
    add_geometry, 
    interpolate_nans,  
    split_metadata_tsdf, 
    drop_non_tokenizable,
    create_tokenized_tsdf, 
    drop_empty_geometries, 
    save_preprocessed_data, 
    load_preprocessed_data
)
from preprocessing.filters import (
    filter_tsdf_by_nans,
    get_metadata_filter,
    filter_tsdf_by_metadata
)
from models.helpers import (
    pt2df,
    configure_torch, 
    get_lstm_configs,
    get_scaled_splits,
    evaluate_performance,
    get_torch_dataloaders
)
from models.lstm import (
    Net,
    train_model,
    inference_model
)
from visualization.plots import plot_forecast
from utils.configs import get_yaml_configs
from utils.logger import get_logger
    


### Configurations
Configurations are loaded from the ```configurations/default.yml``` file. 

In [2]:
configs = get_yaml_configs()

### Logger
Initialize a logger object to keep track of data statistics. 

In [3]:
logger = get_logger(configs)
logger.critical(f"Configs: {configs.items()}")

### Load data
Data is loaded from this [AWS S3 Cloud Storage bucket]("https://s3.eu-central-1.amazonaws.com/floris.calkoen.open.data/outliers.csv"). 

We will load a csv (sample_data.csv) containing all satellite-derived shoreline position data. This data includes both metadata about the sites and the shoreline positions. 

Alternatively cleaned metadata (sites.csv) and time series (time-series.csv) dataframes can be loaded directly. For demonstration purposes we here continue with the raw sample data.   

In [4]:
data  = pd.read_csv(configs['data']['data_sample_url'])
data

KeyError: 'data_sample_url'

Finally we also load a file (outliers.csv) containing information about outliers which were detected by Luijendijk et al. (2018) and later by Kras (2019). The outlier indices are nested in the csv. We therefore first have to explode the dataframe.

In [ ]:
outliers = pd.read_csv(configs['data']['outliers_url'])
outliers['outliers_1_as_int'] = outliers['outliers_1'].apply(str2int)
outliers['outliers_2_as_int'] = outliers['outliers_2'].apply(str2int)
outliers

### Preprocessing
First we optimize the dtypes of the dataframe to reduce its size. 

In [ ]:
data = optimize(data, ignore_features = ['dt', 'dist', 'outliers_1', 'outliers_2'])

Now we will seperate the time-serise data and metadata. First we create a tokenized version of the dataframe; we split the shoreline positions (distance) and dates (dt) from one string into floats. We also drop the non-tokenizable strings; the sites without observations. Finally we seperate the data into two dataframes: time series and metadata. 

In [ ]:
data = create_tokenized_tsdf(data)
data = drop_non_tokenizable(data)
metadata, tsdf = split_metadata_tsdf(data)

Above we can see that the time series data is now saved in an array. However, both the dates and shoreline positions are still kept in nested data structures. Furthermore, the dates are still expressed in decimals. Here we will unnest the dataframe and change the decimals to datetime objects. 

In [ ]:
tsdf = unnesting(tsdf, explode=['dt', 'dist'])
tsdf = format_tsdf(tsdf)

In [ ]:
metadata

Since we are working with geospatial data, we will now convert the Pandas DataFrame to a GeoPandas DataFrame. Such dataframe with also include a geometry column. Meanwhile we will drop all sites without coordinates. 

In [ ]:
metadata = add_geometry(metadata)
metadata = drop_empty_geometries(metadata)

### Filter by metadata
Here we filter the data according to configurations set in the configurations- (here default.yml) file. First we create a list of the transects we want to include into the analysis based on criteria such as whether the beach is sandy; or if the sediment is composited. We use this filter to get an updated time-series dataframe. We will furthermore drop both outliers 1 and 2 according to the outliers information provided in the file outliers.csv 

Finally we will pivot the dataframe so that each of the columns describe shoreline evolution of one of the transects. 

In [ ]:
metadata_filter = get_metadata_filter(metadata, tsdf, configs)
tsdf = filter_tsdf_by_metadata(tsdf, configs, outliers, metadata_filter)
tsdf = pivot_tsdf(tsdf)
tsdf

In [ ]:
tsdf.shape

### Filter by Nan's
Most transects have missing values for one or more years due to clouds, measurement error's or problems with the satellite. However, to reliably use the data for forecasting we need a certain amount of observations. We therefore only include transects with less than 25% nan's. The remaining nan's will be linearly interpolated in both directions. Finally we will save these preprocessed results priorly to forecasting. Data will be saved in pickle-formate to keep the optimized dataframe.

In [ ]:
tsdf, _, _ = filter_tsdf_by_nans(tsdf, configs)
tsdf = interpolate_nans(tsdf)
save_preprocessed_data(tsdf, metadata, configs)

## Forecasting
We now focus on forecasting. Here we only present a basic LSTM network. In the future shorelines application will also other algorithms be implemented. 

In [ ]:
# Optionally directly load data when data preprocessed data has been saved.
configs, tsdf, metadata = load_preprocessed_data(filename="sample_1598259358.pkl")

### PyTorch configurations
First we set the basic configurations of PyTorch, such as a random seed for reproducability. We further get the LSTM model configurations.  

In [ ]:
configure_torch(seed=configs['run']['seed'])
model_configs = get_lstm_configs(tsdf, configs)

### Split and 
We first divide the dataset into train, validation and test partitions. This data is scaled between 0 and 1. Please note that scaling is only performed after splitting the data into train and test partitions in order to avoid information leaking into the test set. The model will be trained on the training set. Then the hyperparameters will be optimized by evaluation on the validation set. Finally, the model performance will be evaluated on the test set. Later we have to re-scale the output values, so we also save the scaler. 

In [ ]:
train, val, test, test_raw, test_scaler = get_scaled_splits(tsdf, ratio=model_configs['split_ratio'])

### Input preparation
PyTorch includes automatic gradient calculations. However, to use these the data should be hold in PyTorch Tensor's. We additionaly load the tensor's into PyTorch's dataloaders for practical purposes. The data in these dataloaders consists of 10 time series per batch. Furthermore we see that the data is now split into 24:9 proportion: the first 24 years will be used for training, while the 9 latest are used for evaluation. 

In [ ]:
dataloaders = get_torch_dataloaders(train, val, test, train_window=model_configs['train_window'],
                                    batch_size=model_configs['batch_size'])


### RNN Model
Here we define a basic RNN network.

In [ ]:
rnn = Net(model_configs['input_size'], model_configs['hidden_size'], model_configs['output_size'],
           model_configs['batch_size'], model_configs['train_window'])
print(rnn)

### Train loop
We train the model for 10 epochs. This is incredibly fast since we only used a small sample of the dataset.

In [ ]:
train_model(rnn, dataloaders, model_configs)

### Inference
We now use the trained model for inference on the test partition. The overall performance is evaluated by several statistics. 

In [ ]:
forecast = inference_model(rnn, dataloaders['test'], model_configs)
forecast = pt2df(forecast, model_configs, test, inverse_scaling=True, test_scaler=test_scaler)
performance = forecast.apply(lambda x: evaluate_performance(test_raw[x.name], x, model_configs),
                                       result_type="expand").T
performance.columns = model_configs['evaluation']
performance

### Show a forecast example. 

In [ ]:
plot_forecast(test_raw, forecast, configs, model_configs, transect_id=model_configs['transect_id'])

### Save the results 

In [ ]:
print(f"\n Performance LSTM: \n{performance.mean()}")
timestamp = int(datetime.timestamp(datetime.now()))
forecast.to_csv(f"output/forecasts/lstm_{timestamp}.csv")